In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("./pure-methods-pdfbox.csv")
df.shape

(1369, 6)

In [3]:
def extract_from_tags(tag, tags):
    search_string = tag + "=[\w]+"
    extracted_tag = re.findall(search_string, tags)[0]
    extracted_tag = re.findall("\=(.*)", extracted_tag)[0]
    if extracted_tag == "true":
        return True
    else:
        return False

In [4]:
cols = ["visibility", "parent-FQN", "method-name", "param-list", "return-type", 
        "local-variables", "conditionals", "multiple-statements", "loops", "parameters", "returns",
        "switches", "ifs", "static", "returns-primitives"]
final_df = pd.DataFrame(columns = cols)
for index, row in df.iterrows():
    final_df.loc[index, 'visibility'] = row['visibility']
    final_df.loc[index, 'parent-FQN'] = row['parent-FQN']
    final_df.loc[index, 'method-name'] = row['method-name']
    final_df.loc[index, 'param-list'] = row['param-list'].lstrip('[').rsplit(']', 1)[0]
    final_df.loc[index, 'return-type'] = row['return-type']
    final_df.loc[index, 'local-variables'] = extract_from_tags("local_variables", str(row['tags']))
    final_df.loc[index, 'conditionals'] = extract_from_tags("conditionals", str(row['tags']))
    final_df.loc[index, 'multiple-statements'] = extract_from_tags("multiple_statements", str(row['tags']))
    final_df.loc[index, 'loops'] = extract_from_tags("loops", str(row['tags']))
    final_df.loc[index, 'parameters'] = extract_from_tags("parameters", str(row['tags']))
    final_df.loc[index, 'returns'] = extract_from_tags("returns", str(row['tags']))
    final_df.loc[index, 'switches'] = extract_from_tags("switches", str(row['tags']))
    final_df.loc[index, 'ifs'] = extract_from_tags("ifs", str(row['tags']))
    final_df.loc[index, 'static'] = extract_from_tags("static", str(row['tags']))
    final_df.loc[index, 'returns-primitives'] = extract_from_tags("returns_primitives", str(row['tags']))

final_df.head()

,visibility,parent-FQN,method-name,param-list,return-type,local-variables,conditionals,multiple-statements,loops,parameters,returns,switches,ifs,static,returns-primitives
0,public,org.apache.fontbox.cff.CIDKeyedType2CharString,getCID,,int,False,False,False,False,False,True,False,False,False,True
1,public,org.apache.xmpbox.type.ArrayProperty,getArrayType,,org.apache.xmpbox.type.Cardinality,False,False,False,False,False,True,False,False,False,False
2,public,org.apache.fontbox.ttf.HeaderTable,getFontRevision,,float,False,False,False,False,False,True,False,False,False,True
3,public,org.apache.fontbox.ttf.VerticalMetricsTable,getAdvanceHeight,int,int,False,False,False,False,True,True,False,True,False,True
4,public,org.apache.pdfbox.text.TextPosition,getHeightDir,,float,False,False,False,False,False,True,False,False,False,True


### Methods that return objects

In [5]:
(final_df["returns-primitives"] == False).sum()

754

### Methods that accept parameters

In [6]:
(final_df["parameters"] == True).sum()

151

### Methods with if statements

In [7]:
(final_df["ifs"] == True).sum()

37

### Methods wtih conditionals ?:

In [8]:
(final_df["conditionals"] == True).sum()

21

### Methods with local variables

In [9]:
(final_df["local-variables"] == True).sum()

40

### Methods with multiple statements

In [10]:
(final_df["multiple-statements"] == True).sum()

52

In [11]:
instrumentation_candidates_df = pd.DataFrame(columns = cols)
instrumentation_candidates_df = final_df[((final_df['multiple-statements'] == True) | 
                                          (final_df['ifs'] == True) |
                                          (final_df['conditionals'] == True) | 
                                          (final_df['parameters'] == True) | 
                                          (final_df['switches'] == True) |
                                          (final_df['loops'] == True) |
                                          (final_df['local-variables'] == True)) &
                                          (final_df['static'] == False)]
print(instrumentation_candidates_df.shape)
instrumentation_candidates_df.to_csv(r'./instrumentation-candidates-pdfbox.csv', index=False)
instrumentation_candidates_df.tail(60)

(137, 15)


,visibility,parent-FQN,method-name,param-list,return-type,local-variables,conditionals,multiple-statements,loops,parameters,returns,switches,ifs,static,returns-primitives
874,protected,org.apache.pdfbox.pdmodel.common.function.type...,applyforInteger,"int, int",int,False,False,False,False,True,True,False,False,False,True
875,protected,org.apache.pdfbox.pdmodel.common.function.type...,applyforInteger,"int, int",int,False,False,False,False,True,True,False,False,False,True
876,protected,org.apache.pdfbox.pdmodel.common.function.type...,applyforInteger,"int, int",int,False,False,False,False,True,True,False,False,False,True
882,private,org.apache.pdfbox.debugger.colorpane.CSSeparation,getIntRepresentation,float,int,False,False,False,False,True,True,False,False,False,True
917,public,org.apache.fontbox.cff.CharStringCommand$Key,hashCode,,int,False,False,True,False,False,True,False,True,False,True
921,private,org.apache.pdfbox.pdmodel.graphics.shading.Ten...,getBernsteinPolynomials,int,double[][],True,False,True,True,True,True,False,False,False,False
922,private,org.apache.fontbox.ttf.GlyphRenderer,midValue,"int, int",int,False,False,False,False,True,True,False,False,False,True
928,private,org.apache.pdfbox.debugger.flagbitspane.Panose...,getLetterformValue,int,java.lang.String,False,False,False,False,True,True,False,False,False,False
936,public,org.apache.fontbox.cff.CharStringCommand$Key,equals,java.lang.Object,boolean,True,False,True,False,True,True,False,True,False,True
939,public,org.apache.fontbox.ttf.VerticalMetricsTable,getTopSideBearing,int,int,False,False,False,False,True,True,False,True,False,True


In [12]:
sorted(instrumentation_candidates_df['parent-FQN'].unique())

['org.apache.fontbox.afm.AFMParser',
 'org.apache.fontbox.cff.CFFParser$Format0FDSelect',
 'org.apache.fontbox.cff.CFFParser$Format3FDSelect',
 'org.apache.fontbox.cff.CFFParser$RangeMapping',
 'org.apache.fontbox.cff.CharStringCommand$Key',
 'org.apache.fontbox.cff.DataInput',
 'org.apache.fontbox.cff.Type2CharStringParser',
 'org.apache.fontbox.cmap.CIDRange',
 'org.apache.fontbox.cmap.CMap',
 'org.apache.fontbox.cmap.CMapParser',
 'org.apache.fontbox.cmap.CodespaceRange',
 'org.apache.fontbox.ttf.CmapSubtable',
 'org.apache.fontbox.ttf.GlyfSimpleDescript',
 'org.apache.fontbox.ttf.GlyphRenderer',
 'org.apache.fontbox.ttf.GlyphSubstitutionTable$CoverageTableFormat2',
 'org.apache.fontbox.ttf.GlyphSubstitutionTable$LookupTypeSingleSubstFormat1',
 'org.apache.fontbox.ttf.GlyphSubstitutionTable$LookupTypeSingleSubstFormat2',
 'org.apache.fontbox.ttf.HorizontalMetricsTable',
 'org.apache.fontbox.ttf.KerningSubtable',
 'org.apache.fontbox.ttf.KerningSubtable$PairData0Format0',
 'org.apach